# 4.1.2 - Ejemplo CLV, Análisis de cliente

![clv](../../images/clv.jpeg)

## Problema (caso de estudio)

### Resumen

**Descripcion de los datos**:  

+ **customer :** `ID del cliente`                        
+ **state :** `estados en US`                      
+ **customer_lifetime_value :** `CLV es el valor economico del cliente para la empresa durante toda su relacion`
+ **response :**  `Respuesta a campañas/llamadas (marketing-engagement)`                   
+ **coverage :** `Tipo de cobertura del cliente`                        
+ **education** `Nivel educativo del cliente`                   
+ **effective_to_date :** `Fecha efectiva`              
+ **employmentstatus :** `Estado laboral del cliente`                
+ **gender :**  `Genero del cliente`                  
+ **income :**  `Ingresos del cliente`
+ **location_code :** `Zona de residencia del cliente`                  
+ **marital_status :** `Estado Civil del cliente`                  
+ **monthly_premium_auto :** `Premium mensual`            
+ **months_since_last_claim :** `Ultima reclamacion del cliente`          
+ **months_since_policy_inception :** `Inicio de la poliza`    
+ **number_of_open_complaints :** `Reclamaciones abiertas`        
+ **number_of_policies :** `Numero de polizas`              
+ **policy_type :** `Tipo de poliza`                     
+ **policy :** `Poliza`                         
+ **renew_offer_type :**  `Oferta de renovacion`                 
+ **sales_channel :**  `Canal de ventas (primer contacto compañia-cliente)`                 
+ **total_claim_amount :** `Monto de la reclamacion`              
+ **vehicle_class :**  `Tipo de vehiculo`                 
+ **vehicle_size :**  `Tamaño del vehiculo`  


**Objetivo:**

Mejorar el CLV a traves del marketing.

In [ ]:
import warnings
warnings.simplefilter('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import pandas as pd
pd.set_option('display.max_columns', None)
plt.rcParams['figure.figsize']=(20, 10)

## Obtener datos

In [ ]:
data=pd.read_csv('../../../data/Marketing-Customer-Analysis.csv')

data.columns=[e.lower().replace(' ', '_') for e in data.columns]

data.head()

## Analisis exploratorio

In [ ]:
data.info(memory_usage='deep')

#### Objetivo

In [ ]:
data.customer_lifetime_value.describe()

In [ ]:
sns.distplot(data.customer_lifetime_value);

In [ ]:
sns.distplot(data.customer_lifetime_value,
             hist_kws={'cumulative': True}, 
             kde_kws={'cumulative': True});

#### a) Numero total de clientes que responden a las campañas de marketing (engagement)

In [ ]:
data.groupby('response').count().customer

In [ ]:
ax=data.groupby('response').count().customer.plot(
        
        kind='bar',
        color=['r', 'b'],
        grid=True,
        figsize=(12, 8),
        title='Engagement'
)

ax.set_xlabel('Engage')
ax.set_ylabel('# Count')

plt.show();

In [ ]:
data.groupby('response').count().customer/data.shape[0]

**Esto nos muestra que solo el 14% de los clientes responden positivamente a las campañas.**

#### b) Oferta de renovacion

Nos fijaremos en los datos de las ofertas de renovacion. Buscaremos que tipo de oferta funciona mejor para los clientes.

In [ ]:
por_tipo_de_oferta=data[data.response=='Yes']\
                        .groupby('renew_offer_type')\
                        .count().customer/data.groupby('renew_offer_type').count().customer


por_tipo_de_oferta

In [ ]:
ax=(por_tipo_de_oferta*100).plot(kind='bar',
                                 figsize=(12, 8),
                                 color=['r', 'g', 'b', 'y'],  
                                 grid=True)

ax.set_xlabel('Tipo de oferta')
ax.set_ylabel('Tasa de engage %');

In [ ]:
data.groupby('renew_offer_type').count().customer

**Podemos ver que la oferta tipo 2 tiene mayor engage.**

#### c) Renovacion/Atributo

Ahora vamos a intentar entender como los clientes con diferentes atributos responden a los mensajes de marketing.
Empezaremos observando la respuesta para cada tipo de oferta y clase de vehiculo.

In [ ]:
def engage(resp, grupo1, grupo2):

    df=data[data.response==resp]\
                .groupby([grupo1, grupo2])\
                .count().customer/data.groupby(grupo1).count().customer
    
    df=df.unstack().fillna(0)

    ax=(df*100).plot(kind='bar',
                     figsize=(12, 8),
                     color=['r', 'g', 'b', 'y', 'grey', 'black'],  
                     grid=True)

    ax.set_ylabel('Tasa de engage %')
    
    plt.show();
    
    return df

In [ ]:
engage('Yes', 'renew_offer_type', 'vehicle_class')

**Ya sabiamos que el tipo 2 de oferta tenia la respuesta mas alta entre los clientes, aunque ahora se pueden ver mas cosas cruzando con otros atributos, por ejemplo, la clase vehiculo. Los clientes con vehiculos de 4 puertas responden con mayor frecuencia a todo tipo de ofertas, y aquellos clientes con Luxury SUV responden con mas alta frecuecia las ofertas tipo 1 y 2. Si tenemos diferencias significativas entre las tasas de respuesta entre los clientes, podemos ajustar el objetivo de las distintas ofertas.**

In [ ]:
data.columns

In [ ]:
engage('Yes', 'renew_offer_type', 'state')

#### d) Analizando la tasas de respuesta por genero

In [ ]:
por_genero=data.loc[data.response=='Yes']\
                    .groupby(['gender'])\
                    .count().customer/data.groupby('gender').count().customer*100.

por_genero

In [ ]:
ax=(por_genero).plot(kind='bar', figsize=(12,8),
                               color=['r', 'g', 'b', 'y'], grid=True)

ax.set_xlabel('Tipo de oferta')
ax.set_ylabel('Tasa Engage %')

plt.show();

In [ ]:
engage('Yes', 'gender', 'vehicle_class')

**Las clases estan balanceadas. Pero se puede observar una menor respuesta entre las mujeres. Por otro lado, los hombres con coches de lujo no responden nunca a las campañas de marketing.**

#### e) Analizando la tasa de respuesta por diferentes canales de venta

In [ ]:
por_canal_venta=data.loc[data.response=='Yes']\
                    .groupby(['sales_channel'])\
                    .count().customer/data.groupby('sales_channel').count().customer

por_canal_venta

In [ ]:
ax=(por_canal_venta*100.).plot(kind='bar', figsize=(12,8),
                               color=['r', 'g', 'b', 'y'], grid=True)


ax.set_ylabel('Tasa Engage %')

plt.show();

**Como podemos ver, Agent funciona mejor en terminos de obtener respuesta de los clientes, y en segundo lugar Web. Podemos ver otras relaciones.**

In [ ]:
engage('Yes', 'sales_channel', 'renew_offer_type')

**La oferta tipo 3 no funciona en el canal Branch ni en el canal Call Center. La oferta tipo 1 necesita mejorar Call Centers.**

#### e) Clusterizacion Sencilla

Vamos a segmentar a los clientes segun el CLV y segun los meses desde el inicio de la poliza.

Aquellos clientes con un CLV mayor que la mediana, se clasifican como alto CLV (`high`), aquellos por debajo de la mediana como bajo CLV (`low`).

In [ ]:
data['clv_segment']=data.customer_lifetime_value\
                        .apply(lambda x: 'high' if x>data.customer_lifetime_value.median() else 'low')

data.head()

In [ ]:
data.months_since_policy_inception.describe()

In [ ]:
data['policy_age_segment']=data.months_since_policy_inception\
                        .apply(lambda x: 'high' if x>data.months_since_policy_inception.median() else 'low')

data.head()

In [ ]:
logy=False

ax=data.loc[(data.clv_segment=='high')&(data.policy_age_segment=='high')]\
       .plot.scatter(x='months_since_policy_inception', y='customer_lifetime_value', 
                    logy=logy, color='red')


data.loc[(data.clv_segment=='low')&(data.policy_age_segment=='high')]\
       .plot.scatter(ax=ax, # con esta variable se pinta uno encima de otro
                     x='months_since_policy_inception', y='customer_lifetime_value', 
                     logy=logy, color='blue')


data.loc[(data.clv_segment=='high')&(data.policy_age_segment=='low')]\
       .plot.scatter(ax=ax,
                     x='months_since_policy_inception', y='customer_lifetime_value', 
                     logy=logy, color='orange')


data.loc[(data.clv_segment=='low')&(data.policy_age_segment=='low')]\
       .plot.scatter(ax=ax,
                     x='months_since_policy_inception', y='customer_lifetime_value', 
                     logy=logy, color='green', grid=True, figsize=(10, 7))


ax.set_ylim(1e3, 5e4)
ax.set_ylabel('CLV (escala log)')
ax.set_xlabel('Meses')
ax.set_title('Segmentos')
plt.show();

`logy=True` transforma la escala una escala logaritmica, que se usa a menudo en valores monetario, aunque suelen generar un problema de simitria (skewness). 

In [ ]:
engage('Yes', 'clv_segment', 'policy_age_segment')

**Podemos ver que el segmento de polizas mas antiguas tiene mayor tasa de respuesta. Esto sugiere que esos clientes que llevan mas tiempo en la compañia responden mejor. De hecho, el mejor segmento de los cuatro en respuesta es el de polizas mas antiguas y bajo CLV. Creando diferentes segmentaciones de los clientes basandonos en sus atributos, podemos tener un mejor entendimiento del comportamiento de los distintos tipos de clientes y, por tanto, desarrollar una mejor estrategia de marketing.**

### Conclusion


+ Objetivo:
    + **Mejorar el CLV** (Media CLV: 8005)
    
+ Insights:
    + Solo el 14% de los clientes responden a campañas de marketing (Mejorar respuesta)
    + La oferta tipo 2 tiene la mayor tasa de respuesta (¿Cual es la diferencia entre las ofertas?)
    + Los clientes con Luxury SUV responden con mayor frecuencia a las ofertas 1 y 2
    + Principalmente enfocado en coches de cuatro puertas 
    + La oferta tipo 3 no es representativa (cambiarla y/o eliminarla, enfoque en coches deportivos y de lujo. Lo mismo con la oferta 4)
    + Las mujeres tienen menor tasa de respuesta 
    + Los hombres con coches de lujo no responden en absoluto
    + La oferta tipo 3 no funciona en los canales de venta branch y call center 
    + La oferta tipo 1 necesita mejorar en call centers 
    + Los clientes mas antiguos responden mejor 